# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [4]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
linreg =LinearRegression()
selector = RFE(linreg, n_features_to_select = 5)
selector = selector.fit(boston_features, boston.target)
selected = boston_features.columns[selector.support_]
X = boston_features[selected]
y = pd.DataFrame(boston.target, columns = ['price'])

In [5]:
X.head()

,NOX,RM,DIS,B,LSTAT
0,0.538,6.575,0.542096,1.000000,-1.275260
1,0.469,6.421,0.623954,1.000000,-0.263711
2,0.469,7.185,0.623954,0.989737,-1.627858
3,0.458,6.998,0.707895,0.994276,-2.153192
4,0.458,7.147,0.707895,1.000000,-1.162114


## Train test split

Perform a train-test-split with a test set of 0.20.

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)

In [9]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [10]:
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [12]:
from sklearn.metrics import mean_squared_error
test_residuals = y_test - y_hat_test
train_residuals = y_train - y_hat_train

test_mse = mean_squared_error(y_test, y_hat_test)
train_mse = mean_squared_error(y_train, y_hat_train)

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [15]:
def kfolds(data, k):
    #Force data as pandas dataframe
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k

    leftovers = num_observations%k
    
    folds = []
    start_obs = 0
    for fold_n in range(1,k+1):
        if fold_n <= leftovers:#for first leftovers number of folds
            #Fold Size will be 1 larger to account for leftovers
            fold =  data.iloc[start_obs : start_obs+fold_size+1] 
            folds.append(fold)
            start_obs +=  fold_size + 1
        else:
            fold =  data.iloc[start_obs : start_obs+fold_size] 
            folds.append(fold)
            start_obs +=  fold_size
            
    return folds 

### Apply it to the Boston Housing Data

In [16]:
# Make sure to concatenate the data again
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [17]:
bos_folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [25]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)

[price    23.444681
dtype: float64, price    22.947166
dtype: float64, price    19.336272
dtype: float64, price    15.222148
dtype: float64, price    22.093827
dtype: float64]
[price    15.645328
dtype: float64, price    16.814797
dtype: float64, price    37.825179
dtype: float64, price    57.85782
dtype: float64, price    31.987524
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
cv_5_results = cross_val_score(linreg, X, y, cv =5, scoring ='neg_mean_squared_error')

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [28]:
print(np.mean(test_errs), np.mean(cv_5_results))

32.02612959835912 -32.02612959835912


##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!